In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import HBox, Layout, Output
from IPython.display import display, Image, Markdown
from scipy import interpolate
from scipy.optimize import least_squares
from scipy.integrate import solve_ivp, trapezoid
import pandas as pd

# Parameteridentifikation

In [ ]:
imag = Image("../../images/hvac.png", width=400)

outL = Output()
outR = Output()

with outL:
    display(imag)
with outR:
    display(Markdown("""
**Ausgangssystem**
\\begin{align*}         
    \\dot{x}(t;\\theta) & = -\\underbrace{\\Big(\\frac{v_{\\mathrm{m}}}{l} + \\frac{\\alpha_{\\mathrm{m}\\infty} A_{\\mathrm{M,m}}}{c_{\\mathrm{p,m}} V_{\\mathrm{m}}} \\Big)}_{\\theta_1} x(t) + \\underbrace{\\frac{\\beta}{c_{\\mathrm{p,m}} V_{\\mathrm{m}}}}_{\\theta_2} u(t) + \\underbrace{\\Big(\\frac{v_{\\mathrm{m}}}{l} + \\frac{\\alpha_{\\mathrm{m}\\infty} A_{\\mathrm{M,m}}}{c_{\\mathrm{p,m}} V_{\\mathrm{m}}} \\Big)}_{\\theta_1} T_{\\infty}\\\\
    y(t) & = x(t)
\\end{align*}
mit den unbekannten Parametern
\\begin{align*}
    \\theta & = \\Big( \\theta_1, \\theta_2\\Big)
\\end{align*}

    """))
cols = HBox([outL, outR], layout=Layout(display='flex', flex_flow='row', justify_content='space-around', align_items='center'))
display(cols)

## Messung

In [ ]:
dfData =  pd.read_csv('messungHVAC.csv')
dfData['DateTime'] = pd.to_timedelta(dfData['time'], unit='s')
dfData.set_index('DateTime', inplace=True)
dfData = dfData.resample('1s').ffill()

### Ruhelage

\begin{align*}         
    0 & = -\Big(\frac{v_{\mathrm{m}}}{l} + \frac{\alpha_{\mathrm{m}\infty} A_{\mathrm{M,m}}}{c_{\mathrm{p,m}} V_{\mathrm{m}}} \Big) \bar{y} + \frac{\beta}{c_{\mathrm{p,m}} V_{\mathrm{m}}} \bar{u} + \Big(\frac{v_{\mathrm{m}}}{l} + \frac{\alpha_{\mathrm{m}\infty} A_{\mathrm{M,m}}}{c_{\mathrm{p,m}} V_{\mathrm{m}}} \Big) T_{\infty}\\
    \bar{y} & = T_{\infty} \text{ für } \bar{u} = 0  
\end{align*}


In [ ]:
bT = dfData['Pipe-T'][0:7].mean()
display(Markdown(rf"""
$\bar{{T}} = {bT}$
"""))
dfData['Pipe-T-Norm'] = dfData['Pipe-T'][:] - bT

In [ ]:
fig = plt.figure(figsize=(10, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1 = plt.subplot(111)
ax1.plot(dfData['time'], dfData['Pipe-T'])

ax1.grid()
ax1.set_xlabel(r'$t$ in s')
ax1.set_ylabel(r'$y$ in °C')

ax11 = ax1.twinx()
ax11.plot(dfData['time'], dfData['Traj y'], 'C2')
ax11.tick_params(axis='y', colors='C2')
ax11.set_ylabel('$u$ in -', color='C2');

## Identifikationsdaten

In [ ]:
dfIdent = dfData.iloc[5:420].copy()
dfIdent.index = dfIdent.index - dfIdent.index[0]
dfIdent['time'] = dfIdent['time'] - dfIdent['time'].iloc[0]

## Systemdefinition

In [ ]:
def nonlinSystem(t, y, th1, th2, Ta):
    dy = th1 * y + th2 * u(t) - th1 * Ta

    return [dy]

tSim = np.linspace(dfIdent['time'].iloc[0], dfIdent['time'].iloc[-1], len(dfIdent['time'].values))
u = interpolate.interp1d(dfIdent['time'], dfIdent['Traj y'], fill_value='extrapolate')

## 1. Variante - grafische Methode

**Linearisiertes System** um die Ruhelage $(\bar{x}, \bar{u})$ als lineares PT$_1$-Glied
\begin{align*}
    \dot{\tilde{y}}(t) & = a \tilde{y}(t) + b \tilde{u}(t) & \Leftrightarrow && T_1 \dot{\tilde{y}}(t) & = \tilde{y}(t) + K_{\mathrm{P}} \tilde{u}(t) 
\end{align*}

In [ ]:
KpV1 = dfIdent['Pipe-T-Norm'].iloc[-1]
i = dfIdent['Pipe-T-Norm'][0:420].searchsorted(KpV1 * 0.63)
T1V1 = dfIdent['time'].iloc[i]
aV1 = - 1 / T1V1
bV1 = - KpV1 * aV1 * 1. / 0.25 # u = [0, 0.25] = [0, 1]

In [ ]:
display(Markdown(rf"""
$K_{{\mathrm{{p}}}} = {KpV1}$, &nbsp; $T_1 = {T1V1}$, &nbsp; $a = {aV1}$, &nbsp; $b = {bV1}$
"""))

In [ ]:
fig = plt.figure(figsize=(10, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1 = plt.subplot(111)
ax1.axvline(T1V1, linewidth=2, color='black')
ax1.scatter(T1V1, KpV1 * 0.63, marker='o', s=100, color='C1')
ax1.axline((T1V1, KpV1), slope=KpV1/T1V1, color='black')

ax1.plot(dfIdent['time'], dfIdent['Pipe-T-Norm'])

ax1.grid()
ax1.set_xlabel(r'$t$ in s')
ax1.set_ylabel(r'$T-\bar{T}$ in °C')

ax11 = ax1.twinx()
ax11.plot(dfIdent['time'], dfIdent['Traj y'], 'C2')
ax11.tick_params(axis='y', colors='C2')
ax11.set_ylabel('$u$ in -', color='C2');

## 2. Variante - diskretes Least-Squares

**Linearisiertes System** um die Ruhelage $(\bar{x}, \bar{u})$ als PT$_1$-Glied in Differenzenform
\begin{align*}
    \tilde{y}_{k+1} & = (1 + \check{a}) \tilde{y}_k + \check{b} \tilde{u}_k, &
    \check{a} & = \Delta t\, a, &
    \check{b} & = \Delta t\, b
\end{align*}
als parameterlineare Darstellung
\begin{align*}
    \mathbf{A} \theta & = \mathbf{y}, &
    \mathbf{A} & = \begin{pmatrix}
        \tilde{y}_0 & \tilde{u}_0 \\
        \vdots & \vdots \\
        \tilde{y}_{N-1} & \tilde{u}_{N-1} \\
    \end{pmatrix}, &
    \mathbf{y} & = \begin{pmatrix}
        \tilde{y}_{1}\\
        \vdots\\
        \tilde{y}_{N}
    \end{pmatrix}, &
    \theta & = (1 + \check{a}, \check{b})^{\intercal}
\end{align*}
und Lösung mittels der Pseudoinversen $\mathbf{A}^{\dagger} = (\mathbf{A}^{\intercal}\mathbf{A})^{-1} \mathbf{A}^{\intercal}$ durch
\begin{align*}
    \theta & = \mathbf{A}^{\dagger} \mathbf{y}
\end{align*}

In [ ]:
yk1 = dfIdent['Pipe-T-Norm'][1:]
yk = dfIdent['Pipe-T-Norm'][0:-1]
uk = dfIdent['Traj y'][0:-1]

In [ ]:
Y = yk1.values
A = np.ones((len(Y), 2))
A[:, 0] = yk.values
A[:, 1] = uk.values

In [ ]:
p = np.linalg.inv(A.T @ A) @ A.T @ Y
aV2 = p[0] - 1
bV2 = p[1]

In [ ]:
display(Markdown(rf"""
$a = {aV2}$, &nbsp; $b = {bV2}$
"""))

## 3. Variante - kontinuierliches Least-Squares

Parameterlineares System
\begin{align*}
    \dot{y}(t) & = (T_{\infty} - y(t)) \theta_1 + u(t) \theta_2 u(t)
\end{align*}
in Darstellung
\begin{align*}
    \mathbf{y} & = \mathbf{A} \theta, &
    \mathbf{A} & = \begin{pmatrix}
        \int_0^T(T_{\infty} - y(t))^2 \mathrm{d} t & 
        \int_0^T(T_{\infty} - y(t)) u(t) \mathrm{d} t \\
        \int_0^T(T_{\infty} - y(t)) u(t) \mathrm{d} t & 
        \int_0^Tu^2(t) \mathrm{d} t\\
    \end{pmatrix}, &
    \mathbf{y} & = \begin{pmatrix}
        \int_0^T(T_{\infty} - y(t)) \dot{y}(t) \mathrm{d} t\\
        \int_0^Tu(t) \dot{y}(t) \mathrm{d} t
    \end{pmatrix}, &
    \theta & = (\theta_1, \theta_2)^{\intercal}
\end{align*}
und Lösung mittels
\begin{align*}
    \theta & = \mathbf{A}^{-1} \mathbf{y}
\end{align*}
durch Regularisierung aller Messsignale $y_{\varphi}=y \star \varphi$, $u_{\varphi}=u \star \varphi$ und Schätzung der Ableitung durch $\dot{y}_{\varphi}=y \star \dot{\varphi}$

In [ ]:
_u = dfIdent['Traj y']
_y = dfIdent['Pipe-T']
_Ta = dfIdent['Pipe-T'].iloc[0]

In [ ]:
hFilter = 10
tFil = np.linspace(0, hFilter, hFilter + 1)
regFunc = lambda t, h, k: (t / h) ** k * (1 - t / h) ** k
dregFunc = lambda t, h, k: k * (t / h) ** k * ((h - t) / h) ** k * (h - 2 * t) / (t * (h -t))
regFil = regFunc(tFil, hFilter, 2)
dregFil = dregFunc(tFil, hFilter, 2)
dregFil[0] = dregFil[-1] = 0

In [ ]:
uFilter = np.zeros(len(tSim))
yFilter = np.zeros(len(tSim))
yDiffFilter = np.zeros(len(tSim))
for idx, _t in enumerate(tSim[hFilter:-hFilter]):
    uFilter[idx+hFilter] = trapezoid(y=_u[idx:idx+hFilter+1] * regFil,
                                     x=tSim[idx:idx+hFilter+1]) / tSim[1] / np.sum(regFil)
    yFilter[idx+hFilter] = trapezoid(y=_y[idx:idx+hFilter+1] * regFil,
                                     x=tSim[idx:idx+hFilter+1]) / tSim[1] / np.sum(regFil)
    yDiffFilter[idx+hFilter] = trapezoid(y=_y[idx:idx+hFilter+1] * dregFil,
                                         x=tSim[idx:idx+hFilter+1]) / tSim[1] ** 2 / -np.sum(regFil)

In [ ]:
S = np.array([[
    trapezoid(y=(-_Ta + yFilter[hFilter:-hFilter]) ** 2, x=tSim[hFilter:-hFilter]),
    trapezoid(y=(-_Ta + yFilter[hFilter:-hFilter]) * uFilter[hFilter:-hFilter], x=tSim[hFilter:-hFilter]),
], [
    trapezoid(y=(-_Ta + yFilter[hFilter:-hFilter]) * uFilter[hFilter:-hFilter], x=tSim[hFilter:-hFilter]),
    trapezoid(y=uFilter[hFilter:-hFilter] ** 2, x=tSim[hFilter:-hFilter]),
]])
y = np.array([
    [trapezoid(y=(-_Ta + yFilter[hFilter:-hFilter]) * yDiffFilter[hFilter:-hFilter], x=tSim[hFilter:-hFilter])],
    [trapezoid(y=uFilter[hFilter:-hFilter] * yDiffFilter[hFilter:-hFilter], x=tSim[hFilter:-hFilter]),]
])
p = np.linalg.inv(S) @ y            
aV3 = p[0, 0]
bV3 = p[1, 0]

## 4. Variante - Optimierung

**System**
\begin{align*}         
    \dot{y}(t) & = -\Big(\frac{v_{\mathrm{m}}}{l} + \frac{\alpha_{\mathrm{m}\infty} A_{\mathrm{M,m}}}{c_{\mathrm{p,m}} V_{\mathrm{m}}} \Big) y(t) + \frac{\beta}{c_{\mathrm{p,m}} V_{\mathrm{m}}} u(t) + \Big(\frac{v_{\mathrm{m}}}{l} + \frac{\alpha_{\mathrm{m}\infty} A_{\mathrm{M,m}}}{c_{\mathrm{p,m}} V_{\mathrm{m}}} \Big) T_{\infty}\\
    & =  - \theta_1 y(t) + \theta_2 u(t) + \theta_1 T_{\infty}
\end{align*}

In [ ]:
def optSys(p):
    y0 = [dfIdent['Pipe-T'].iloc[0]]
    res = solve_ivp(nonlinSystem,
                    [tSim[0], tSim[-1]],
                    y0,
                    t_eval=tSim, 
                    args=(p[0], p[1], dfIdent['Pipe-T'].iloc[0]))
    return dfIdent['Pipe-T'].values - res.y[0]

In [ ]:
y0 = np.array([0.01, 0.01])
resLq = least_squares(optSys, y0)
aV4 = resLq.x[0]
bV4 = resLq.x[1]

## Validierung

In [ ]:
display(Markdown(rf"""
|                     | grafisch  | diskretes LQ  | kontinuierliches LQ | Optimierung |
| :---                | :----:      | :----:      | :---:       | ---:        |
| $a$                 | {aV1}       | {aV2}       | {aV3}       | {aV4}       |
| $b$                 | {bV1}       | {bV2}       | {bV3}       | {bV4}       |
"""))

In [ ]:
y0 = [dfIdent['Pipe-T'].iloc[0]]
resV1 = solve_ivp(nonlinSystem,
                  [tSim[0], tSim[-1]],
                  y0,
                  t_eval=tSim, 
                  args=(aV1, bV1, dfIdent['Pipe-T'].iloc[0]))
resV2 = solve_ivp(nonlinSystem,
                  [tSim[0], tSim[-1]],
                  y0,
                  t_eval=tSim, 
                  args=(aV2, bV2, dfIdent['Pipe-T'].iloc[0]))
resV3 = solve_ivp(nonlinSystem,
                  [tSim[0], tSim[-1]],
                  y0,
                  t_eval=tSim, 
                  args=(aV3, bV3, dfIdent['Pipe-T'].iloc[0]))
resV4 = solve_ivp(nonlinSystem,
                  [tSim[0], tSim[-1]],
                  y0,
                  t_eval=tSim, 
                  args=(aV4, bV4, dfIdent['Pipe-T'].iloc[0]))

In [ ]:
fig = plt.figure(figsize=(12, 6))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)
ax1 = plt.subplot(111)
ax1.plot(dfIdent['time'], dfIdent['Pipe-T'], label='Original')
ax1.plot(resV1.t, resV1.y.T, label='grafisch')
ax1.plot(resV2.t, resV2.y.T, '-.', label='diskretes LQ')
ax1.plot(resV3.t, resV3.y.T, '--', label='kont. LQ')
ax1.plot(resV4.t, resV4.y.T, ':', label='Optimierung')

ax1.grid()
ax1.set_xlabel(r'$t$ in s')
ax1.set_ylabel(r'$y$ in °C')

handlesAx1, labelsAx1 = ax1.get_legend_handles_labels()
ax1.legend([handle for i, handle in enumerate(handlesAx1)],
           [label for i, label in enumerate(labelsAx1)],
           bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=5, mode="expand", borderaxespad=0., framealpha=0.5)

ax11 = ax1.twinx()
ax11.plot(dfIdent['time'], dfIdent['Traj y'], 'C2')
ax11.tick_params(axis='y', colors='C2')
ax11.set_ylabel(r'$u$ in -', color='C2')
plt.show()